In [1]:
#  Pandas library
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Import the pandas library
import pandas as pd

# Define the file path for the large dataset
file_path = "C:\\Users\\Azalas12\\Desktop\\E-commerce Customer Data For Behavior Analysis\\ecommerce_customer_data_large.csv"

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first 5 rows to confirm it loaded correctly
print("First 5 rows of the dataset:")
print(df.head())

First 5 rows of the dataset:
   Customer ID        Purchase Date Product Category  Product Price  Quantity  \
0        44605  2023-05-03 21:30:02             Home            177         1   
1        44605  2021-05-16 13:57:44      Electronics            174         3   
2        44605  2020-07-13 06:16:57            Books            413         1   
3        44605  2023-01-17 13:14:36      Electronics            396         3   
4        44605  2021-05-01 11:29:27            Books            259         4   

   Total Purchase Amount Payment Method  Customer Age  Returns Customer Name  \
0                   2427         PayPal            31      1.0   John Rivera   
1                   2448         PayPal            31      1.0   John Rivera   
2                   2345    Credit Card            31      1.0   John Rivera   
3                    937           Cash            31      0.0   John Rivera   
4                   2598         PayPal            31      1.0   John Rivera   

   

In [15]:
# Data Cleaning and Preparation
# Check for a comprehensive overview of the DataFrame, including data types and non-null values
# This helps us quickly identify columns with missing data
print("--- DataFrame Information ---")
df.info()

--- DataFrame Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Customer ID            250000 non-null  int64  
 1   Product Category       250000 non-null  object 
 2   Product Price          250000 non-null  int64  
 3   Quantity               250000 non-null  int64  
 4   Total Purchase Amount  250000 non-null  int64  
 5   Payment Method         250000 non-null  object 
 6   Customer Age           250000 non-null  int64  
 7   Returns                250000 non-null  float64
 8   Customer Name          250000 non-null  object 
 9   Age                    250000 non-null  int64  
 10  Gender                 250000 non-null  object 
 11  Churn                  250000 non-null  int64  
 12  recency                250000 non-null  int64  
 13  frequency              250000 non-null  int64  
 14  moneta

In [5]:
# Check for duplicate rows across the entire dataset
# The .sum() method will count the number of True values, telling us how many duplicates there are
print("\n--- Number of Duplicate Rows ---")
print(f"Total duplicate rows: {df.duplicated().sum()}")


--- Number of Duplicate Rows ---
Total duplicate rows: 0


In [7]:
# Fill missing values in the 'Returns' column with 0
# The 'inplace=True' argument modifies the DataFrame directly without creating a new one
df['Returns'] = df['Returns'].fillna(0)

# Confirm that the missing values have been filled
# The .isnull().sum() method is a quick way to count missing values per column
print("--- Missing Values After Imputation ---")
print(df.isnull().sum())

--- Missing Values After Imputation ---
Customer ID              0
Purchase Date            0
Product Category         0
Product Price            0
Quantity                 0
Total Purchase Amount    0
Payment Method           0
Customer Age             0
Returns                  0
Customer Name            0
Age                      0
Gender                   0
Churn                    0
dtype: int64


In [8]:
# Convert the 'Purchase Date' column to a datetime object
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Confirm the change by checking the DataFrame information again
print("--- DataFrame Information After Date Conversion ---")
df.info()

--- DataFrame Information After Date Conversion ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Customer ID            250000 non-null  int64         
 1   Purchase Date          250000 non-null  datetime64[ns]
 2   Product Category       250000 non-null  object        
 3   Product Price          250000 non-null  int64         
 4   Quantity               250000 non-null  int64         
 5   Total Purchase Amount  250000 non-null  int64         
 6   Payment Method         250000 non-null  object        
 7   Customer Age           250000 non-null  int64         
 8   Returns                250000 non-null  float64       
 9   Customer Name          250000 non-null  object        
 10  Age                    250000 non-null  int64         
 11  Gender                 250000 non-null  object      

In [9]:
# Calculate (RECENCY) the most recent date in the dataset to use as a reference point
current_date = df['Purchase Date'].max() + pd.Timedelta(days=1)

# Group the DataFrame by 'Customer ID' and find the maximum (most recent) 'Purchase Date' for each customer.
# Then, subtract that date from our 'current_date' to get the recency in days.
recency_df = df.groupby('Customer ID')['Purchase Date'].max().reset_index()
recency_df['recency'] = (current_date - recency_df['Purchase Date']).dt.days

# Display the new DataFrame with customer ID and recency
print("Recency DataFrame:")
print(recency_df.head())

Recency DataFrame:
   Customer ID       Purchase Date  recency
0            1 2022-11-29 06:48:25      289
1            2 2023-07-03 17:26:19       73
2            3 2023-02-03 03:58:07      223
3            4 2022-06-29 03:41:09      442
4            5 2022-07-16 04:08:09      425


In [10]:
# Calculating FREQUENCY and MONETARY VALUE 
# Group by 'Customer ID' and use a dictionary inside .agg() to apply different aggregations to different columns.
# We'll count the number of rows for Frequency and sum the 'Total Purchase Amount' for Monetary Value.
rfm_df = df.groupby('Customer ID').agg(
    frequency=('Customer ID', 'size'),
    monetary_value=('Total Purchase Amount', 'sum')
).reset_index()

# Display the new DataFrame with Frequency and Monetary Value
print("\nFrequency and Monetary Value DataFrame:")
print(rfm_df.head())


Frequency and Monetary Value DataFrame:
   Customer ID  frequency  monetary_value
0            1          3            6290
1            2          6           16481
2            3          4            9423
3            4          5            7826
4            5          5            9769


In [11]:
# Merge the recency and rfm DataFrames based on 'Customer ID'
master_df = pd.merge(recency_df, rfm_df, on='Customer ID')

# Display the final master DataFrame with all the new features
print("\nMaster DataFrame with Recency, Frequency, and Monetary Value:")
print(master_df.head())


Master DataFrame with Recency, Frequency, and Monetary Value:
   Customer ID       Purchase Date  recency  frequency  monetary_value
0            1 2022-11-29 06:48:25      289          3            6290
1            2 2023-07-03 17:26:19       73          6           16481
2            3 2023-02-03 03:58:07      223          4            9423
3            4 2022-06-29 03:41:09      442          5            7826
4            5 2022-07-16 04:08:09      425          5            9769


In [12]:
# Merge the 'master_df' with the original 'df' DataFrame
# We'll use a 'left' merge to keep all the original rows from 'df'
# The 'on' argument specifies the common column to merge on
df = pd.merge(df, master_df[['Customer ID', 'recency', 'frequency', 'monetary_value']], on='Customer ID', how='left')

# Drop the 'Purchase Date' from the merged master_df as it is now redundant
df.drop('Purchase Date', axis=1, inplace=True)

# Display the first 5 rows of the final DataFrame to show the new columns
print("Final DataFrame after merging new features:")
print(df.head())

Final DataFrame after merging new features:
   Customer ID Product Category  Product Price  Quantity  \
0        44605             Home            177         1   
1        44605      Electronics            174         3   
2        44605            Books            413         1   
3        44605      Electronics            396         3   
4        44605            Books            259         4   

   Total Purchase Amount Payment Method  Customer Age  Returns Customer Name  \
0                   2427         PayPal            31      1.0   John Rivera   
1                   2448         PayPal            31      1.0   John Rivera   
2                   2345    Credit Card            31      1.0   John Rivera   
3                    937           Cash            31      0.0   John Rivera   
4                   2598         PayPal            31      1.0   John Rivera   

   Age  Gender  Churn  recency  frequency  monetary_value  
0   31  Female      0      133          5           10

In [13]:
# Exploratory Data Analysis (EDA)
# Comparing the average recency, frequency, and monetary_value for customers who churned versus those who did not
# Group the DataFrame by the 'Churn' column and calculate the mean for each group
# We'll focus on our three new features: recency, frequency, and monetary_value
eda_results = df.groupby('Churn')[['recency', 'frequency', 'monetary_value']].mean()

# Display the results
print("Average RFM values for Churned (1) vs. Non-Churned (0) customers:")
print(eda_results)

Average RFM values for Churned (1) vs. Non-Churned (0) customers:
          recency  frequency  monetary_value
Churn                                       
0      218.226953   5.991695    16321.589413
1      216.314881   6.008298    16373.152982


In [16]:
# Digging deeper
# Group the DataFrame by 'Product Category' and calculate the mean of the 'Churn' column.
# The mean of a 0/1 (no churn/churn) column gives us the churn rate.
category_churn_rate = df.groupby('Product Category')['Churn'].mean().reset_index()

# Sort the results in descending order to easily see which categories have the highest churn rate
category_churn_rate.sort_values(by='Churn', ascending=False, inplace=True)

# Display the results
print("Churn Rate by Product Category:")
print(category_churn_rate)

Churn Rate by Product Category:
  Product Category     Churn
1         Clothing  0.201227
0            Books  0.200797
2      Electronics  0.200431
3             Home  0.199626


In [17]:
# Digging more deep for insights → Gender and Churn & Payment Method and Churn
# Group the DataFrame by 'Gender' and calculate the mean of the 'Churn' column.
gender_churn_rate = df.groupby('Gender')['Churn'].mean().reset_index()

# Sort the results
gender_churn_rate.sort_values(by='Churn', ascending=False, inplace=True)
print("Churn Rate by Gender:")
print(gender_churn_rate)

print("\n-----------------------\n")

# Group the DataFrame by 'Payment Method' and calculate the mean of the 'Churn' column.
payment_method_churn_rate = df.groupby('Payment Method')['Churn'].mean().reset_index()

# Sort the results
payment_method_churn_rate.sort_values(by='Churn', ascending=False, inplace=True)
print("Churn Rate by Payment Method:")
print(payment_method_churn_rate)

Churn Rate by Gender:
   Gender     Churn
0  Female  0.201626
1    Male  0.199426

-----------------------

Churn Rate by Payment Method:
  Payment Method     Churn
0           Cash  0.202525
2         PayPal  0.202430
1    Credit Card  0.196620


In [18]:
# Going even deeper through age grouping against churn
# Define age ranges for our bins
age_bins = [18, 25, 35, 45, 55, 65, 75]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-74']

# Create a new column 'Age_Group' by binning the 'Customer Age' column
df['Age_Group'] = pd.cut(df['Customer Age'], bins=age_bins, labels=labels, right=False)

# Group by the new 'Age_Group' and calculate the mean churn rate for each group
age_group_churn = df.groupby('Age_Group')['Churn'].mean().reset_index()

# Sort the results in descending order to easily see which age groups have the highest churn rate
age_group_churn.sort_values(by='Churn', ascending=False, inplace=True)

print("Churn Rate by Age Group:")
print(age_group_churn)

Churn Rate by Age Group:
  Age_Group     Churn
0     18-24  0.210725
5     65-74  0.205667
4     55-64  0.201225
2     35-44  0.198850
1     25-34  0.198557
3     45-54  0.192921


C:\Users\Azalas12\AppData\Local\Temp\ipykernel_26708\805273779.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_churn = df.groupby('Age_Group')['Churn'].mean().reset_index()


In [21]:
# Create bins for 'monetary_value'
monetary_bins = [0, 5000, 10000, 20000, 50000, 100000]
monetary_labels = ['0-5K', '5K-10K', '10K-20K', '20K-50K', '50K+']
df['Monetary_Group'] = pd.cut(df['monetary_value'], bins=monetary_bins, labels=monetary_labels, right=False)

# Create bins for 'frequency' - Fixed to ensure monotonic increase
max_freq = df['frequency'].max()
# Check if max frequency is greater than our last defined bin
if max_freq <= 20:
    # If max is 20 or less, adjust the bins to ensure they increase
    frequency_bins = [0, 5, 10, 15, max_freq]
    frequency_labels = ['1-5', '6-10', '11-15', '16+']
else:
    # Original bins if max frequency is greater than 20
    frequency_bins = [0, 5, 10, 15, 20, max_freq]
    frequency_labels = ['1-5', '6-10', '11-15', '16-20', '20+']

df['Frequency_Group'] = pd.cut(df['frequency'], bins=frequency_bins, labels=frequency_labels, right=False)

# Group by the three features and calculate the mean of 'Churn'
combined_analysis = df.groupby(['Product Category', 'Monetary_Group', 'Frequency_Group'])['Churn'].mean().reset_index()

# Sort the results by churn rate to see the highest and lowest combinations
combined_analysis.sort_values(by='Churn', ascending=False, inplace=True)

print("Top 10 Combinations by Churn Rate:")
print(combined_analysis.head(10))

print("\nBottom 10 Combinations by Churn Rate:")
print(combined_analysis.tail(10))

Top 10 Combinations by Churn Rate:
   Product Category Monetary_Group Frequency_Group     Churn
18            Books           50K+           11-15  1.000000
78             Home           50K+           11-15  1.000000
58      Electronics           50K+           11-15  1.000000
38         Clothing           50K+           11-15  1.000000
10            Books        10K-20K           11-15  0.306011
30         Clothing        10K-20K           11-15  0.300493
50      Electronics        10K-20K           11-15  0.263441
61             Home           0-5K            6-10  0.214286
4             Books         5K-10K             1-5  0.207668
24         Clothing         5K-10K             1-5  0.206669

Bottom 10 Combinations by Churn Rate:
   Product Category Monetary_Group Frequency_Group  Churn
56      Electronics           50K+             1-5    NaN
57      Electronics           50K+            6-10    NaN
62             Home           0-5K           11-15    NaN
63             Home    

C:\Users\Azalas12\AppData\Local\Temp\ipykernel_26708\938175152.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  combined_analysis = df.groupby(['Product Category', 'Monetary_Group', 'Frequency_Group'])['Churn'].mean().reset_index()


In [22]:
# Save the final DataFrame to a CSV file
df.to_csv('customer_churn_data.csv', index=False)

In [24]:
import os

os.path.exists('customer_churn_data.csv')

True